In [ ]:
"""
Multiple Correspondence Analysis (MCA) - Obesity Dataset
Approach 2: Supplementary quantitative variables

This notebook performs MCA only on categorical variables, then projects
quantitative variables (Age and Weight) as supplementary variables.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings

warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("Set2")

In [ ]:
# Load data
df = pd.read_csv("dataset/obesity.csv")

# Select variables
categorical_vars = [
    "Gender",
    "family_history_with_overweight",
    "FAVC",
    "CAEC",
    "MTRANS",
]

quantitative_vars = ["Age", "Weight"]
target_var = "NObeyesdad"

# Extract subset
df_subset = df[categorical_vars + quantitative_vars + [target_var]].copy()

print(f"Dataset: {len(df_subset)} observations")
print(f"Active categorical variables: {len(categorical_vars)}")
print(f"Supplementary quantitative variables: {len(quantitative_vars)}")

In [ ]:
# Descriptive statistics
print("\nCategorical variables:")
for var in categorical_vars:
    print(f"\n{var}:")
    counts = df_subset[var].value_counts()
    for category, count in counts.items():
        print(f"  {category}: {count} ({count/len(df_subset)*100:.1f}%)")

print("\nQuantitative variables:")
print(df_subset[quantitative_vars].describe().round(2))

In [ ]:
# Create indicator matrix (only categorical)
df_categorical = df_subset[categorical_vars].copy()
df_indicators = pd.get_dummies(df_categorical, drop_first=False, dtype=int)
print(
    f"\nIndicator matrix: {df_indicators.shape[0]} rows × {df_indicators.shape[1]} columns"
)

# Standardize indicator matrix
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_indicators)

# Apply MCA (PCA on indicator matrix)
n_components = 10
mca = PCA(n_components=n_components)
X_mca = mca.fit_transform(X_scaled)

# Variance explained
print("\nVariance explained:")
cumulative = 0
for i, var in enumerate(mca.explained_variance_ratio_[:5], 1):
    cumulative += var
    print(f"Dimension {i}: {var*100:.2f}% (cumulative: {cumulative*100:.2f}%)")

In [ ]:
# Scree plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.bar(
    range(1, n_components + 1),
    mca.explained_variance_ratio_ * 100,
    color="steelblue",
    alpha=0.8,
)
ax1.set_xlabel("Dimension", fontsize=10)
ax1.set_ylabel("Variance explained (%)", fontsize=10)
ax1.set_title("Scree plot", fontsize=11, fontweight="bold")
ax1.grid(axis="y", alpha=0.3)

cumulative = np.cumsum(mca.explained_variance_ratio_ * 100)
ax2.plot(
    range(1, n_components + 1),
    cumulative,
    marker="o",
    linewidth=2,
    markersize=7,
    color="darkred",
)
ax2.axhline(y=80, color="green", linestyle="--", linewidth=1.5, alpha=0.7)
ax2.set_xlabel("Number of dimensions", fontsize=10)
ax2.set_ylabel("Cumulative variance (%)", fontsize=10)
ax2.set_title("Cumulative variance", fontsize=11, fontweight="bold")
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig("approach2_scree.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Individuals projection
fig, ax = plt.subplots(figsize=(12, 8))

obesity_codes = pd.Categorical(df_subset[target_var]).codes
scatter = ax.scatter(
    X_mca[:, 0],
    X_mca[:, 1],
    c=obesity_codes,
    cmap="viridis",
    alpha=0.6,
    s=35,
    edgecolors="k",
    linewidth=0.2,
)

ax.axhline(y=0, color="gray", linewidth=0.8, alpha=0.5)
ax.axvline(x=0, color="gray", linewidth=0.8, alpha=0.5)
ax.set_xlabel(f"Dimension 1 ({mca.explained_variance_ratio_[0]*100:.2f}%)", fontsize=10)
ax.set_ylabel(f"Dimension 2 ({mca.explained_variance_ratio_[1]*100:.2f}%)", fontsize=10)
ax.set_title(
    "Individuals projection (colored by obesity level)", fontsize=11, fontweight="bold"
)
ax.grid(alpha=0.3)

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label("Obesity level", fontsize=10)

plt.tight_layout()
plt.savefig("approach2_individuals.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Variable categories contributions
loadings = mca.components_.T
contributions_df = pd.DataFrame(
    loadings[:, :3], columns=["Dim1", "Dim2", "Dim3"], index=df_indicators.columns
)

print("\nTop 10 contributors to Dimension 1:")
for cat, contrib in (
    contributions_df["Dim1"].abs().sort_values(ascending=False).head(10).items()
):
    print(f"  {cat}: {contrib:.3f}")

print("\nTop 10 contributors to Dimension 2:")
for cat, contrib in (
    contributions_df["Dim2"].abs().sort_values(ascending=False).head(10).items()
):
    print(f"  {cat}: {contrib:.3f}")

In [ ]:
# Categories projection
fig, ax = plt.subplots(figsize=(12, 12))

circle = plt.Circle((0, 0), 1, color="navy", fill=False, linewidth=2)
ax.add_patch(circle)

# Select top categories for display
top_categories = set(
    list(contributions_df["Dim1"].abs().nlargest(10).index)
    + list(contributions_df["Dim2"].abs().nlargest(10).index)
)

for category in top_categories:
    x = contributions_df.loc[category, "Dim1"]
    y = contributions_df.loc[category, "Dim2"]

    ax.arrow(
        0,
        0,
        x,
        y,
        head_width=0.03,
        head_length=0.03,
        fc="red",
        ec="red",
        linewidth=1.5,
        alpha=0.7,
    )

    ax.text(
        x * 1.15,
        y * 1.15,
        category,
        fontsize=8,
        ha="center",
        va="center",
        bbox=dict(
            boxstyle="round,pad=0.3",
            facecolor="yellow",
            alpha=0.7,
            edgecolor="black",
            linewidth=0.5,
        ),
    )

ax.set_xlim(-1.3, 1.3)
ax.set_ylim(-1.3, 1.3)
ax.axhline(y=0, color="gray", linewidth=0.8, alpha=0.5)
ax.axvline(x=0, color="gray", linewidth=0.8, alpha=0.5)
ax.set_xlabel(f"Dimension 1 ({mca.explained_variance_ratio_[0]*100:.2f}%)", fontsize=10)
ax.set_ylabel(f"Dimension 2 ({mca.explained_variance_ratio_[1]*100:.2f}%)", fontsize=10)
ax.set_title("Variable categories projection", fontsize=11, fontweight="bold")
ax.grid(alpha=0.3)
ax.set_aspect("equal")

plt.tight_layout()
plt.savefig("approach2_categories.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Project supplementary quantitative variables
print("\nSupplementary quantitative variables projection:")

# Standardize quantitative variables
quant_scaler = StandardScaler()
quant_scaled = quant_scaler.fit_transform(df_subset[quantitative_vars])

# Calculate correlations with MCA dimensions
correlations = np.zeros((len(quantitative_vars), n_components))
for i, var in enumerate(quantitative_vars):
    for j in range(n_components):
        correlations[i, j] = np.corrcoef(quant_scaled[:, i], X_mca[:, j])[0, 1]

corr_df = pd.DataFrame(
    correlations[:, :3], columns=["Dim1", "Dim2", "Dim3"], index=quantitative_vars
)

print("\nCorrelations with dimensions:")
print(corr_df.round(3))

In [ ]:
# Visualize supplementary variables
fig, ax = plt.subplots(figsize=(10, 10))

circle = plt.Circle((0, 0), 1, color="navy", fill=False, linewidth=2)
ax.add_patch(circle)

for var in quantitative_vars:
    x = corr_df.loc[var, "Dim1"]
    y = corr_df.loc[var, "Dim2"]

    ax.arrow(
        0,
        0,
        x,
        y,
        head_width=0.05,
        head_length=0.05,
        fc="blue",
        ec="blue",
        linewidth=2.5,
        alpha=0.8,
    )

    ax.text(
        x * 1.2,
        y * 1.2,
        var,
        fontsize=11,
        fontweight="bold",
        ha="center",
        va="center",
        bbox=dict(
            boxstyle="round,pad=0.5",
            facecolor="lightblue",
            alpha=0.9,
            edgecolor="darkblue",
            linewidth=1.5,
        ),
    )

ax.set_xlim(-1.3, 1.3)
ax.set_ylim(-1.3, 1.3)
ax.axhline(y=0, color="gray", linewidth=0.8, alpha=0.5)
ax.axvline(x=0, color="gray", linewidth=0.8, alpha=0.5)
ax.set_xlabel(f"Dimension 1 ({mca.explained_variance_ratio_[0]*100:.2f}%)", fontsize=10)
ax.set_ylabel(f"Dimension 2 ({mca.explained_variance_ratio_[1]*100:.2f}%)", fontsize=10)
ax.set_title("Supplementary quantitative variables", fontsize=11, fontweight="bold")
ax.grid(alpha=0.3)
ax.set_aspect("equal")

plt.tight_layout()
plt.savefig("approach2_supplementary.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Combined visualization: categories + supplementary variables
fig, ax = plt.subplots(figsize=(12, 12))

circle = plt.Circle((0, 0), 1, color="navy", fill=False, linewidth=2)
ax.add_patch(circle)

# Plot categorical variables (red)
for category in top_categories:
    x = contributions_df.loc[category, "Dim1"]
    y = contributions_df.loc[category, "Dim2"]

    ax.arrow(
        0,
        0,
        x,
        y,
        head_width=0.02,
        head_length=0.02,
        fc="red",
        ec="red",
        linewidth=1,
        alpha=0.6,
    )

    ax.text(
        x * 1.12,
        y * 1.12,
        category,
        fontsize=7,
        ha="center",
        va="center",
        bbox=dict(
            boxstyle="round,pad=0.2",
            facecolor="yellow",
            alpha=0.6,
            edgecolor="black",
            linewidth=0.3,
        ),
    )

# Plot quantitative variables (blue)
for var in quantitative_vars:
    x = corr_df.loc[var, "Dim1"]
    y = corr_df.loc[var, "Dim2"]

    ax.arrow(
        0,
        0,
        x,
        y,
        head_width=0.04,
        head_length=0.04,
        fc="blue",
        ec="blue",
        linewidth=2,
        alpha=0.9,
    )

    ax.text(
        x * 1.25,
        y * 1.25,
        var,
        fontsize=10,
        fontweight="bold",
        ha="center",
        va="center",
        bbox=dict(
            boxstyle="round,pad=0.4",
            facecolor="lightblue",
            alpha=0.9,
            edgecolor="darkblue",
            linewidth=1.2,
        ),
    )

ax.set_xlim(-1.4, 1.4)
ax.set_ylim(-1.4, 1.4)
ax.axhline(y=0, color="gray", linewidth=0.8, alpha=0.5)
ax.axvline(x=0, color="gray", linewidth=0.8, alpha=0.5)
ax.set_xlabel(f"Dimension 1 ({mca.explained_variance_ratio_[0]*100:.2f}%)", fontsize=10)
ax.set_ylabel(f"Dimension 2 ({mca.explained_variance_ratio_[1]*100:.2f}%)", fontsize=10)
ax.set_title(
    "Combined projection: categories (red) + quantitative (blue)",
    fontsize=11,
    fontweight="bold",
)
ax.grid(alpha=0.3)
ax.set_aspect("equal")

# Add legend
from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor="yellow", edgecolor="black", label="Categorical variables"),
    Patch(facecolor="lightblue", edgecolor="darkblue", label="Quantitative variables"),
]
ax.legend(handles=legend_elements, loc="upper right", fontsize=9)

plt.tight_layout()
plt.savefig("approach2_combined.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Summary
print("\nSummary:")
print(
    f"  Total variance (first 2 dims): {mca.explained_variance_ratio_[:2].sum()*100:.2f}%"
)
print(
    f"  Total variance (first 3 dims): {mca.explained_variance_ratio_[:3].sum()*100:.2f}%"
)
print(f"  Active categorical variables: {len(categorical_vars)}")
print(f"  Indicator variables: {len(df_indicators.columns)}")
print(f"  Supplementary variables: {len(quantitative_vars)}")
print(f"\nInterpretation:")
print(f"  Weight correlation with Dim1: {corr_df.loc['Weight', 'Dim1']:.3f}")
print(f"  Age correlation with Dim1: {corr_df.loc['Age', 'Dim1']:.3f}")